In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
CSV_dataset_URL = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
dataset_df = pd.read_csv(CSV_dataset_URL)
dataset_columns = dataset_df.columns

In [ ]:
print(dataset_df.dtypes)
print(dataset_df)

In [ ]:
number_columns = ['Age', 'Fare']
categorical_columns = ['Sex', 'Embarked', 'Survived', 'Pclass', 'Parch', 'SibSp']

In [ ]:
# Clean data
dataset_df.drop(['Cabin', 'Name'], axis = 1, inplace = True)
dataset_df.dropna(axis = 0, inplace = True)
print(dataset_df)

In [ ]:
# boxplots of number columns
dataset_df.boxplot('Age', grid = True, figsize = (2, 5))

In [ ]:
dataset_df.boxplot('Fare', grid = True, figsize = (2, 10))

In [ ]:
# Density plots for number columns
axis = dataset_df['Age'].plot.hist(density = True, xlim = [-20, 90], bins = range(1, 80))
axis.set_xlabel('Age in years')
dataset_df['Age'].plot.density(ax = axis)
plt.show()

In [ ]:
axis = dataset_df['Fare'].plot.hist(density = True, xlim = [-100, 100], bins = range(1, 100))
axis.set_xlabel('Fare in $')
dataset_df['Fare'].plot.density(ax = axis)
plt.show()

In [ ]:
# TODO: plots for binary/categorical columns